In [35]:
import numpy as np 
import pandas as pd
import sklearn
# from sklearn.externals import joblib
from gensim.models.word2vec import Word2Vec
import gensim.models.word2vec as word2vec

DIM = 100


In [36]:
''' load and init''' 
train_path = 'train.csv'
data = pd.read_csv(train_path)
train_label = data.score
train_text = data.text
eval_path = "evaluation.csv"
eval_data = pd.read_csv(eval_path)
eval_text = eval_data.text
eval_label = eval_data.score
test_path = "test.csv"
test_data = pd.read_csv(test_path)
test_text = test_data.text
test_label = test_data.score

In [37]:
'''pre-process data''' 
import nltk
# nltk.download()
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
stops = set(stopwords.words("english"))
# stops = set()

def comment_to_words(text):
    
    text = BeautifulSoup(text).get_text()

    words = re.sub("[^a-zA-Z]", " ", text)
    # print(words)
    words = words.lower().split()
    # print(words)
    words = [w for w in words if not w in stops]

    return words

In [38]:
''' get corpus input data'''
from tqdm import tqdm
n_comments = train_text.size
clean_comments = []

eval_word_comments = []

test_word_comments = []
for i in tqdm(range(len(test_text))):
    test_word_comments.append(comment_to_words(test_text[i]))
for i in tqdm(range(len(eval_text))):
    eval_word_comments.append(comment_to_words(eval_text[i]))
for i in tqdm(range(n_comments)):
    clean_comments.append(comment_to_words(train_text[i]))
clean_comments += test_word_comments + eval_word_comments

100%|██████████| 7500/7500 [00:01<00:00, 4671.44it/s]


In [39]:
word_vec = Word2Vec(sentences=clean_comments,vector_size=DIM) # get word vector from corpus comments


In [40]:
'''function to calculate the mean of a sentence'''
def build_vec(sent,size,w2v):
    vec = np.zeros(size).reshape((1,size))
    count = 0
    for w in sent:
        # print(w)
        try:
            vec += w2v.wv[w].reshape((1,size))
            count += 1

        except:
            continue
    if count!=0:
        vec /= count
    # print(vec)
    
    return vec

In [41]:
''' calculate all sentence vector and divide the corpus to train test and eval'''
corpus_vec = np.concatenate([build_vec(text,DIM,word_vec) for text in clean_comments])
train_vec = corpus_vec[:7500]

In [42]:
from sklearn.svm import SVC

import joblib
clf = SVC()
clf.fit(train_vec, train_label)


joblib.dump(clf,'svc_model.pkl')


['svc_model.pkl']

In [46]:

clf = joblib.load('svc_model.pkl')

test_vec = np.concatenate([build_vec(test_text,DIM,word_vec) for test_text in test_word_comments])

print('accuracy of test dataset is:',clf.score(test_vec, test_label))

accuracy of test dataset is: 0.7636


In [47]:

eval_vec = np.concatenate([build_vec(eval_text,DIM,word_vec) for eval_text in eval_word_comments])
print('accuracy of evaluation dataset is:',clf.score(eval_vec, eval_label))

accuracy of evaluation dataset is: 0.7646
